# モデルの評価


## 全体的評価（モデルの適合度の評価）

### カイ2乗検定

最尤法で母数を推定した場合、$\chi^2$検定を行うことができる。

標本数が十分に大きいとき、

$$
\chi^2 = f_{ML} \times (N-1)
$$

は自由度$df = \frac{1}{2} p (p + 1) - q$の$\chi^2$分布に近似的に従う（$p$は観測変数の数、$q$は自由母数の数）

「構成されたモデルは正しい」という帰無仮説のもとで検定を行う。

ただし、サンプル数が大きいほど棄却されやすくなる（「モデルが正しいとはいえない」という結果になる）ため、あまり信頼性のない方法

カイ2乗検定の結果は直接使わないが、カイ2乗値を使う別の指標があることもあって統計ソフトでもいまだに計算される

### CFI（comparative fit index）

- 独立モデル（independent model）と呼ばれる最も単純なモデルと比較し、そこからモデルとデータの乖離がどれだけ改善されたかを測る
- 独立モデルは観測変数間に一切パスを引かないモデル

$$
CFI = 1 - \frac{ \chi^2_{分析モデル} - df_{分析モデル} }{ \chi^2_{独立モデル} - df_{独立モデル} }
$$

独立モデルの自由度は

$$
\DeclareMathOperator{\tr}{\text{tr}}
\DeclareMathOperator{\diag}{\text{diag}}
\begin{align}
\chi^2_{独立モデル} &= \tr( \diag(S)^{-1} S) - \log |(\diag(S))^{-1} S| - p\\
                    &= p - \log |(\diag(S))^{-1} S| - p\\
                    &= - \log |(\diag(S))^{-1} S|\\
df_{独立モデル} &= \frac{1}{2} p(p - 1)
\end{align}
$$


似たものでNFIやTLI（NNFI）もある

$$
NFI = \frac{ \chi^2_{独立モデル} - \chi^2_{分析モデル} }{ \chi^2_{独立モデル} }
= 1 - \frac{ \chi^2_{分析モデル} }{ \chi^2_{独立モデル} }
$$

- NFIの$\chi^2$の部分を$\chi^2 - df$にしたものがCFI
- NFIの$\chi^2$の部分を$\chi^2 / df$にして分母から1を引いたものがTLI

$$
TLI = \frac{ (\chi^2_{独立モデル} / df_{独立モデル}) - (\chi^2_{分析モデル} / df_{分析モデル}) }{ (\chi^2_{独立モデル}/df_{独立モデル}) - 1 }
$$

### RMSEA（root mean square error of approximation）

モデルの分布と真の分布との乖離度を1自由度あたりの値として評価する

$$
RMSEA = \sqrt{
    \max
    \left(
        \frac{ \chi^2 }{ df \times n }
        - \frac{1}{n}
       , 0
    \right)
}
$$

RMSEAは0.05以下であれば当てはまりがよく、0.1以上であれば当てはまりがわるいと判断する慣習がある。

### 情報量基準

$$
\begin{align}
AIC &= \chi^2 - 2\times df\\
BIC &= \chi^2 - \log(n) \times df
\end{align}
$$


### GFI（goodness of fit index）

実測値の分散$S$を予測値の分散$\Sigma$で説明できた比率を表す。回帰分析における決定係数に相当する。

$$
\DeclareMathOperator{\tr}{\text{tr}}
GFI = 1 - \frac{ \tr( [\Sigma(\hat{\theta})^{-1}(S - \Sigma(\hat{\theta})) ]^2 ) }{ \tr([\Sigma(\hat{\theta})^{-1} S ]^2) }
= 1 - \frac{ \tr( [\Sigma(\hat{\theta})^{-1} S - I ]^2 ) }{ \tr([\Sigma(\hat{\theta})^{-1} S ]^2) }
$$


- 長所：標本数にあまり影響を受けない
- 短所：モデルの自由度が大きくなると、値が大きくなりにくくなる → 複雑なモデルを作ったほうが有利になってしまう


### AGFI（adjusted goodness of fit index）

$$
AGFI = 1 - \frac{p(p+1)}{2df} (1 - GFI)
$$

- GFIの自由度のバイアスを補正したもの。回帰分析における自由度調整済み決定係数に相当する。
- $AGFI\leq GFI$という関係がある



### RMR（root mean square residual）

実測値の分散と予測値の分散の二乗誤差を測る。機械学習におけるRMSE（root mean squared error）に相当する

$$
RMR = \sqrt{
    \frac{2}{p(p+1)}
    \sum_{j=1}^p \sum_{l=1}^j
    (s_{jl} - \hat{\sigma}_{jl})^2
}
$$

### SRMR（standardized root mean square residual）

単位の影響を除くために正規化したもの。

$$
SRMR = \sqrt{
    \frac{2}{p(p+1)}
    \sum_{j=1}^p \sum_{l=1}^j
    \left(
        \frac{ s_{jl} }{\sqrt{s_{jj} - s_{ll}}}
        - \frac{ \hat{\sigma}_{jl} }{ \sqrt{ \hat{\sigma}_{jj} - \hat{\sigma}_{ll} } }
    \right)^2
}
$$


## 部分的評価（母数の推定値の評価）

### $z$検定

「母数の真の値は0である」という帰無仮説のもとで、

$$
z = \frac{ 推定値 }{ 標準誤差 }
$$

は$df = 観測変数の数n - 自由母数の数q$の$z$分布（標準正規分布）に従う

### 標準化残差による観測変数の評価

$$
残差行列 = S - \hat{\Sigma}
$$

残差行列の各要素の大きさ（絶対値）を確認することで、大きな残差になってい部分はモデルが標本共分散行列をうまく説明できていないことがわかる

実際には標準化した残差を用いる

$$
\frac{s_{jl} - \hat{\sigma}_{jl}}
{\sqrt{(s_{jj}s_{ll} + s_{jl}^2) / n }}
$$

### 決定係数

各変数によって説明された分散の割合

## 参考文献

- [星野崇宏, 岡田謙介, & 前田忠彦. (2005). 構造方程式モデリングにおける適合度指標とモデル改善について: 展望とシミュレーション研究による新たな知見. 行動計量学, 32(2), 209-235.](https://www.jstage.jst.go.jp/article/jbhmk/32/2/32_2_209/_pdf)